In [9]:
import json

In [ ]:
category_keywords = {
        "daily life": ["home", "food", "eat", "shop", "family", "house", 
        "daily","routine", "pet"],
        "work": ["job", "employee", "office", "business", "company", "salary",
        "work"],
        "social life": ["friend", "party", "relationship", "social", "talk", 
        "meet", "chat"],
        "education": ["school", "university", "study", "learn", "education", 
        "class", "teacher"],
        "travel": ["travel", "trip", "flight", "airport", "hotel", "tourism",
        "vacation"],
        "health": ["hospital", "doctor", "medicine", "disease", "clinic"],
}

In [11]:
def classify_words(jsonl_data, category_keywords): 
    definitions_texts = []
    
    definitions = jsonl_data.get("definitions", [])
    for entry in definitions:
        for definition in entry.get("definitions", []):
            definitions_texts.append(definition.lower())
            
    for category, keywords in category_keywords.items():
        if any(keyword in definition for definition in definitions_texts
            for keyword in keywords):
            return category
        
    return "other"


In [12]:
classified_words = {}

with open("datasets/words-de02e1507605431abd5d829d7e868af5.jsonl", "r", 
        encoding="utf-8") as file:
    for line in file:
        word_entry = json.loads(line)
        for word, word_data in word_entry.items():
            category = classify_words(word_data, category_keywords)
            classified_words[word] = category
            
for word, category in classified_words.items():
    print(f"{word}: {category}")


dictionary: work
free: other
thesaurus: other
encyclopedia: work
portmanteau: daily life
cat: daily life
word: other
book: work
pound: daily life
GDP: other
pond: other
nonsense: other
pie: other
crow: daily life
raven: daily life
elephant: daily life
brown: daily life
December: other
month: other
January: other
February: other
march: daily life
April: other
may: other
June: other
July: other
august: other
September: other
October: other
November: other
multiculturalism: social life
day: work
Monday: other
Tuesday: other
Wednesday: other
Thursday: other
Friday: work
Saturday: other
Sunday: travel
lexicography: social life
antonym: other
connotation: other
denotation: other
synonym: other
dialect: daily life
hyponym: education
semantics: daily life
false friend: social life
noun: other
hour: work
